In [4]:
# import re
# import collections
# import pdfplumber
# import os
# import json

# from classBook import Book


# dataDir = "../data/"
# dataName = "Deep Learning.pdf"

In [4]:
import json
import os
import re

import pdfplumber

dataDir = "../data/"
dataName = "Deep Learning.pdf"


class Book:
    num_pages = -1
    book_end_page = "735"
    page_offset = 15

    def __init__(self, name=dataName, dataDir=dataDir):
        self.name = name
        self.dataDir = dataDir
        self.pdf = self.loader()

    def loader(self):
        print("Reading book: ", self.name, "from directory: ", self.dataDir)
        try:
            pdf = pdfplumber.open(os.path.join(self.dataDir, self.name))
        except:
            print("Error: File not found")
            return None
        self.num_pages = len(pdf.pages)
        print("Book loaded successfully")
        print("Number of pages: ", self.num_pages)
        return pdf

    def close(self):
        self.pdf.close()

    def readPage(self, page=-1):
        if page == -1:
            return self.pdf.pages
        else:
            return self.pdf.pages[page - 1]

    def readPageInInterval(self, start_page, end_page, offset=page_offset):
        return self.pdf.pages[start_page - 1 + offset : end_page - 1 + offset]

    def searchStrInPage(self, page, str):
        page_text = self.readPage(page)
        return page_text.extract_text().lower().find(str.lower())

    def getToc(self):
        pages = []
        for i in range(8):
            if self.searchStrInPage(i, "Contents") != -1:
                pages.append(i)
        # reg expression to match '6 Deep Feedforward Networks 168'
        pattern_chapter = re.compile(r"(\d+)\s+(.*)\s+(\d+)")
        # match '6.1 Example: Learning XOR . . . . . . . . . . . . . . . . . . . . . . . 171',
        pattern_section = re.compile(
            r"(\d+)\.(\d+)\s+([\?\,\'\’\(\)a-zA-Z\:\s\-]+)\s+.*\s+(\d+)"
        )

        # save to dict
        toc = {}
        for page in pages:
            page_text = self.readPage(page)
            text = page_text.extract_text()
            lines = text.split("\n")
            for line in lines:
                match_chapter = pattern_chapter.match(line)
                match_section = pattern_section.match(line)
                if match_chapter:
                    chapter = {
                        "chapter": match_chapter.group(1),
                        "title": match_chapter.group(2),
                        "page": match_chapter.group(3),
                    }
                elif match_section:
                    section = {
                        "chapter": match_section.group(1),
                        "section": match_section.group(2),
                        "title": match_section.group(3),
                        "page": match_section.group(4),
                    }
                    if chapter["chapter"] not in toc:
                        toc[chapter["chapter"]] = {
                            "title": chapter["title"],
                            "page": chapter["page"],
                            "sections": [],
                        }
                    toc[chapter["chapter"]]["sections"].append(section)

        # add end page
        for chapter in toc:
            try:
                toc[chapter]["end_page"] = toc[str(int(chapter) + 1)]["page"]
            except:
                toc[chapter]["end_page"] = self.book_end_page
            for section in toc[chapter]["sections"]:
                try:
                    section["end_page"] = toc[chapter]["sections"][
                        int(section["section"])
                    ]["page"]
                except:
                    section["end_page"] = toc[chapter]["end_page"]
        # write to json
        with open(os.path.join(self.dataDir, "toc.json"), "w") as f:
            json.dump(toc, f, indent=4)
        return len(toc)

    def loadToc(self):
        with open(os.path.join(self.dataDir, "toc.json"), "r") as f:
            toc = json.load(f)
        return toc

    def getChapter(self, chapter: str):
        toc = self.loadToc()
        page = toc[chapter]["page"]
        end_page = toc[chapter]["end_page"]
        return self.readPageInInterval(int(page), int(end_page))

    # read by sections, since we need to extract the relations between section and entity.
    def getSection(self, chapter: str, section: str):
        # transform section number to index
        section_idx = int(section) - 1
        toc = self.loadToc()
        page = toc[chapter]["sections"][section_idx]["page"]
        end_page = toc[chapter]["sections"][section_idx]["end_page"]
        return self.readPageInInterval(int(page), int(end_page))


book = Book()

Reading book:  Deep Learning.pdf from directory:  ../data/
Book loaded successfully
Number of pages:  800


In [7]:
import math

import IPython
import torch
import wikipedia
from pyvis.network import Network
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [8]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

# Specify the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50272, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50272, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [9]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = "", "", "", ""
    text = text.strip()
    current = "x"
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = "t"
            if relation != "":
                relations.append(
                    {
                        "head": subject.strip(),
                        "type": relation.strip(),
                        "tail": object_.strip(),
                    }
                )
                relation = ""
            subject = ""
        elif token == "<subj>":
            current = "s"
            if relation != "":
                relations.append(
                    {
                        "head": subject.strip(),
                        "type": relation.strip(),
                        "tail": object_.strip(),
                    }
                )
            object_ = ""
        elif token == "<obj>":
            current = "o"
            relation = ""
        else:
            if current == "t":
                subject += " " + token
            elif current == "s":
                object_ += " " + token
            elif current == "o":
                relation += " " + token
    if subject != "" and relation != "" and object_ != "":
        relations.append(
            {"head": subject.strip(), "type": relation.strip(), "tail": object_.strip()}
        )
    return relations

In [10]:
class KB:
    def __init__(self):
        self.entities = {}
        self.relations = set()
        self.raw_relations = set()

    def get_wikipedia_data(self, candidate_entity):
        try:
            page = wikipedia.page(candidate_entity, auto_suggest=False)
            entity_data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary,
            }
            return entity_data
        except:
            return None

    def print(self):
        print("Entities:")
        for e in self.entities:
            print(f"  {e} -> {self.entities[e]}")
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

    def add_entity(self, e):
        self.entities[e["title"]] = {k: v for k, v in e.items() if k != "title"}

    def add_relation(self, r):
        triplet = (r["head"], r["type"], r["tail"])
        self.raw_relations.add(triplet)
        # check on wikipedia
        candidate_entities = [r["head"], r["tail"]]
        entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

        # if one entity does not exist, stop
        if any(ent is None for ent in entities):
            return

        # manage new entities
        for e in entities:
            self.add_entity(e)

        # rename relation entities with their wikipedia titles
        r["head"] = entities[0]["title"]
        r["tail"] = entities[1]["title"]

        # manage new relation
        triplet = (r["head"], r["type"], r["tail"])
        self.relations.add(triplet)

        # print(f"Added relation: {triplet}")
        # print(self.relations)
        # raise Exception and exit
        # raise Exception("here")

In [11]:
def from_small_text_to_kb(text, verbose=False, max_length=512):
    """This function takes a text and returns a KB. Note that the text should be small enough to fit in the model's input.
    Specifically, the text should be less than 512 tokens, which corresponds to about 380 English words.

    Args:
        text (_type_): text used to generate the KB.
        verbose (bool, optional): If True, prints the number of tokens in the text. Defaults to False.
    Returns:
        KB: Knowledge base generated from the text.
    """

    kb = KB()

    # Tokenizer text

    model_inputs = tokenizer(
        text, max_length=max_length, padding=True, truncation=True, return_tensors="pt"
    ).to(device)

    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate

    gen_kwargs = {
        "max_length": 512,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3,
    }

    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )

    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb

    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)

        for r in relations:
            kb.add_relation(r)

    return kb

In [12]:
def from_text_to_kb(text, span_length=128, verbose=False):
    """This function takes a text and returns a KB. There is no limit on the text length.
    The text is split into spans of length span_length, and a KB is generated for each span.

    Args:
        text (_type_): text used to generate the KB.
        span_length (int, optional): length of each span. Defaults to 128.
        verbose (bool, optional): Defaults to False.

    Returns:
        KB: Knowledge base generated from the text.
    """

    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")

    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append(
            [start + span_length * i, start + span_length * (i + 1)]
        )
        start -= overlap
    if verbose:
        print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [
        inputs["input_ids"][0][boundary[0] : boundary[1]]
        for boundary in spans_boundaries
    ]
    tensor_masks = [
        inputs["attention_mask"][0][boundary[0] : boundary[1]]
        for boundary in spans_boundaries
    ]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks),
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences,
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {"spans": [spans_boundaries[current_span_index]]}
            kb.add_relation(relation)
        i += 1

    return kb

In [13]:
entityDir = os.path.join(dataDir, "entities")
if not os.path.exists(entityDir):
    os.makedirs(entityDir)
relationDir = os.path.join(dataDir, "relations")
if not os.path.exists(relationDir):
    os.makedirs(relationDir)
rawRelationDir = os.path.join(dataDir, "raw_relations")
if not os.path.exists(rawRelationDir):
    os.makedirs(rawRelationDir)

In [14]:
def from_chapter_to_kb(chapter_str: str, verbose=False):
    print("Generating KB for chapter", chapter_str)

    chapter = book.getChapter(chapter_str)

    kb = KB()
    for page in chapter:
        text = page.extract_text()

        kb_page = from_small_text_to_kb(text, verbose=verbose, max_length=1024)
        kb.entities.update(kb_page.entities)
        kb.relations.update(kb_page.relations)
        kb.raw_relations.update(kb_page.raw_relations)

    with open(os.path.join(entityDir, f"kb_chapter_{chapter_str}.json"), "w") as f:
        json.dump(kb.entities, f, indent=4)
    with open(os.path.join(relationDir, f"kb_chapter_{chapter_str}.json"), "w") as f:
        json.dump(list(kb.relations), f, indent=4)
    with open(os.path.join(rawRelationDir, f"kb_chapter_{chapter_str}.json"), "w") as f:
        json.dump(list(kb.raw_relations), f, indent=4)

    print(f"number of entities in chapter {chapter}: {len(kb.entities)}")
    print(f"number of relations in chapter {chapter}: {len(kb.relations)}")
    print(f"number of raw relations in chapter {chapter}: {len(kb.raw_relations)}")

    return kb

In [15]:
chapters = [str(i) for i in range(1, 21)]

for chapter in chapters:
    # from_chapter_to_kb(chapter, 1)
    pass

In [16]:
def read_chapter_by_sections(chapter, save=False):
    toc = book.loadToc()

    res = {}
    for section in toc[chapter]["sections"]:
        print(f"Reading chapter {chapter}, section {section['section']}")
        section_text = [
            page.extract_text().replace("\n", " ")
            for page in book.getSection(chapter, section["section"])
        ]
        kb = from_small_text_to_kb(section_text, verbose=True)
        res[
            section["chapter"] + "." + section["section"] + " " + section["title"]
        ] = kb.relations

    if save:
        with open(os.path.join(dataDir, f"chapter_{chapter}.json"), "w") as f:
            json.dump(res, f, indent=4)

    return res

In [17]:
# count the data file
import os
import json

dataDir = "../data/"
rawRelationDir = os.path.join(dataDir, "raw_relations")
files = os.listdir(rawRelationDir)

data = set()
for file in files:
    with open(os.path.join(rawRelationDir, file), "r") as f:
        data.update(set(map(tuple, json.load(f))))

print(len(data))

# extract entities from raw relations
entities = set()
for relation in data:
    entities.add(relation[0])
    entities.add(relation[2])
print(len(entities))

cacheDir = os.path.join(dataDir, "cache")
if not os.path.exists(cacheDir):
    os.makedirs(cacheDir)

# save relations and entities to cache
with open(os.path.join(cacheDir, "relations.json"), "w") as f:
    json.dump(list(data), f, indent=4)
with open(os.path.join(cacheDir, "entities.json"), "w") as f:
    json.dump(list(entities), f, indent=4)

1914
1302


In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Chapter 1 Introduction Inventors have long dreamed of creating machines that think. This desire dates back to at least the time of ancient Greece. The mythical figures Pygmalion, Daedalus, and Hephaestus may all be interpreted as legendary inventors, and Galatea, Talos, and Pandora may all be regarded as artificial life (Ovid and Martin, 2004; Sparkes, 1996; Tandy, 1997). When programmable computers were first conceived, people wondered whether such machines might become intelligent, over a hundred years before one was built (Lovelace, 1842). Today, artificial intelligence (AI) is a thriving field with many practical applications and active research topics. We look to intelligent software to automate routine labor, understand speech or images, make diagnoses in medicine and support basic scientific research. In the early days of artificial intelligence, the field rapidly tackled and solved problems that are intellectually difficult for human beings but relatively straight- forward for 

In [2]:
# Load model directly
from transformers import AutoModel, AutoTokenizer
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

The model 'BertModel' is not supported for ner. Supported models are ['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GPT2ForTokenClassification', 'GPT2ForTokenClassification', 'GPTBigCodeForTokenClassification', 'GPTNeoForTokenClassification', 'GPTNeoXForTokenClassification', 'IBertForTokenClassification', 'LayoutLMForTokenClassification', 'LayoutLMv2ForTokenC

In [ ]:
test_text = "".join(
    [page.extract_text().replace("\n", " ") for page in book.getChapter("1")[:1]]
)[:200]

print(test_text)

# example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(test_text)
for result in ner_results:
    print(result)

In [ ]:
# import spacy

# test_text = "".join(
#     [page.extract_text().replace("\n", " ") for page in book.getChapter("1")]
# )

# # Load the English model
# nlp = spacy.load("en_core_web_sm", disable=["parser"])

# # Process a text
# doc = nlp(test_text)

# # Iterate over the entities
# for ent in doc.ents:
#     # Print the entity text and its label
#     print(ent.text, ent.label_)